In [1]:
using Plots;plotly()
using Optim
using Zygote
using Meteor.Utility
using Meteor.ExactDiag
using DataFrames;using CSV
using Pkg

In [2]:
using LinearAlgebra
ccall((:openblas_get_num_threads64_, Base.libblas_name),Cint,())
LinearAlgebra.BLAS.set_num_threads(8)

In [3]:
using Base.Threads;nthreads()

7

In [10]:
function thermal_state(dims,nbar)
    c::Array{Complex{Float64}}=zeros(dims,dims)
    if nbar==0
        c[1,1]=1
    else
        for i in 1:dims
           c[i,i]= exp(-i*log(1/nbar+1))
        end
    end
    c=c/tr(c)
    return c        
end

function fock_state(dims,n)
    c::Matrix{ComplexF64}=zeros(dims,dims)
    if n<dims
        c[n+1,n+1]=1
    end
    return c
end

function eye(n)
    k::Array{Complex{Float64}}=zeros(n,n)
    for i in 1:n
        k[i,i]=1
    end
    return k    
end


function a₋(n)      #n为维数
    s::Array{Complex{Float64}}=zeros(n,n)
    for i=1:n-1
        s[i,i+1]=sqrt(i)
    end
    return s
end

a₊(n)=adjoint(a₋(n));

dim=10
ν=1
dot_num=30
initial_state=kron([1.0 0 0;0 0 0;0 0 0],thermal_state(dim,1))
#########密度矩阵向量化，超算子维数变成n^2*n^2维,态矩阵厄米，只用存一半？意义不大
function dm2vec(dm)#态向量化
#julia里是按列reshape的,reshape成列向量
    return reshape(dm,:,1)
end

function vec2dm(vec)
    return reshape(vec,3*dim,:)
end


n=kron(eye(3),a₊(dim)*a₋(dim))
en=kron([0 0 0;0 0 0;0 0 1],eye(dim))
η1=0.15
η2=-0.15
a=a₊(dim)+a₋(dim)
ag2=kron([0 0 0;0 0 0;1 0 0],exp(im*η1*a))
ag3=kron([0 0 1;0 0 0;0 0 0],exp(-im*η1*a))
ag4=ag2+ag3
ar2=kron([0 0 0;0 0 0;0 1 0],exp(im*η2*a))
ar3=kron([0 0 0;0 0 1;0 0 0],exp(-im*η2*a))
ar4=ar2+ar3

E=eye(3*dim)
γ1=40/3*ν
γ2=20/3*ν

function diss1()
    dtheta = 0.01
    x=zeros((3*dim)^2,(3*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*η1*theta*a)
        aup=kron([0 0 0;0 0 0;1 0 0],jump')
        adown=kron([0 0 1;0 0 0;0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end
function diss2()
    dtheta = 0.01
    x=zeros((3*dim)^2,(3*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*η2*theta*a)
        aup=kron([0 0 0;0 0 0;0 1 0],jump')
        adown=kron([0 0 0;0 0 1;0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end

dissipation1=γ1*diss1()
dissipation2=γ2*diss2()

# Δg=(Ωr^2+Ωg^2)/ν-ν
# Δr=Δg
instate=dm2vec(initial_state);
ttot=300

300

In [5]:
function eit_cooling(d, Delta, nu, eta_g, omega_g, eta_r, omega_r, gamma_g, gamma_r, state, t)
    ps = nlevel(["g", "r", "e"])
    pb = boson(d=d)
    
    xop = pb["a"] + pb["adag"]
    
    ham = Lindbladian([ps, pb])
    add_unitary!(ham, (1,), ("e<-e",), coeff=-Delta)
    add_unitary!(ham, (2,), ("n",), coeff=nu)
    
    exp_xop = exp(im*eta_g*xop)
    add_unitary!(ham, (1, 2), ("e<-g", exp_xop), coeff=omega_g/2)
    add_unitary!(ham, (1, 2), ("g<-e", exp_xop'), coeff=omega_g/2)

    exp_xop = exp(im*eta_r*xop)
    add_unitary!(ham, (1, 2), ("e<-r", exp_xop), coeff=omega_r/2)
    add_unitary!(ham, (1, 2), ("r<-e", exp_xop'), coeff=omega_r/2)
    
    dtheta = 0.01
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*eta_g*theta*xop)
        add_dissipation!(ham, (1, 2), ("g<-e", jump), coeff=gamma_g/2)
    end
    
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*eta_r*theta*xop)
        add_dissipation!(ham, (1, 2), ("r<-e", jump), coeff=gamma_r/2)
    end
    
    observer = EDObservers(ham)
    add!(observer, (1,), ("e<-e",), name="e")
    add!(observer, (1,), ("g<-g",), name="g")
    add!(observer, (1,), ("r<-r",), name="r")
    add!(observer, (2,), ("n",), name="n")
    
    L = size(state, 1)

    ham = immrep(ham)
    ham = matrix(ham)
    state = reshape(state, L*L)
    
    observer = immrep(observer)
    result = measure(observer, reshape(state, L, L))
    obs = Observables(result)
    mt = t/100
    n = round(Int, t/mt)
    for tj in 1:n
        state = evolve(ham, mt, state, ishermitian=false)
        tmp = measure(observer, reshape(state, L, L))
        append!(obs, tmp)
    end
    
    return obs
end


eit_cooling (generic function with 1 method)

In [8]:
ttot1=300
function plotme()    
    plot(yticks=0:0.2:1)
    plot!(0:ttot1/100:ttot1,eit_cooling(dim, 111.9645147569779, ν, η1, 3.6103223565556357, η2, 21.05702368447422, γ1, γ2, kron(thermal_state(dim,1),[1.0 0 0;0 0 0;0 0 0]), ttot1)["n[2]"])
end
plotme()#用来做对比

<!DOCTYPE html>
 
 
 Plots.jl

In [9]:
#探究张硕师兄的方案的优化效果，这里上图就是参照，至少要比上图好，那么总时间也是300；
#两段为一组，对于EIT冷却，观察激发态布居始终为0，所以内态布居看成始终是g态和r态之间的转变，大概30个单位时间g和r之间完成一次跃迁
#第二段脉冲失谐量为0，是r和e之间的转换脉冲，先简单的设置成和第一个脉冲时间相等吧，都设置成,试试
#暂且不管脉冲重复的问题，先优化一段试试；
#时间一共60(τ)，分为4段，EIT，空，re,空，循环两次，

In [98]:
function eit_Lindbladian_vec(Ωg,Ωr,Δ)
    unitary::Matrix{ComplexF64}=kron(E,-im*(ν*n-Δ*en+Ωg/2*ag4+Ωr/2*ar4))+kron(transpose(im*((ν*n-Δ*en+Ωg/2*ag4+Ωr/2*ar4))),E)
    
    return unitary+dissipation1+dissipation2
end

function eit_Lindbladian_vec_wait()
    unitary::Matrix{ComplexF64}=kron(E,-im*(ν*n))+kron(transpose(im*((ν*n))),E)
    
    return unitary+dissipation1+dissipation2
end
h_wait=eit_Lindbladian_vec_wait()

function aftereit_vec(Ωr)
    unitary::Matrix{ComplexF64}=kron(E,-im*(ν*n+Ωr/2*ar4))+kron(transpose(im*((ν*n+Ωr/2*ar4))),E)
    
    return unitary+dissipation1+dissipation2
end

τ=60
function feval(x)#x=[1.omegag, 2.omegar, 3.Delta1,  4.t1, 5.Omega_r2],t1,t2分别是两次脉冲的时间,tw是t1和t2之前的等待时间
    f=exp(x[4]*eit_Lindbladian_vec(x[1],x[2],x[3]))*instate
    f=exp((τ-x[4])*aftereit_vec(x[5]))*f
    
    f=exp(x[4]*eit_Lindbladian_vec(x[1],x[2],x[3]))*f
    f=exp((τ-x[4])*aftereit_vec(x[5]))*f   
    
    a=abs(real(tr(n*vec2dm(f))))
    print(a," ")
    return a
end

function g!(G,seq)
    print("g!  ")
    a=real(Zygote.gradient(feval,seq)[1])
    for i in 1:length(seq)
       G[i]=a[i] 
    end
end


g! (generic function with 1 method)

In [72]:
#加约束区间,为啥只能用IPNewton？,哎，不太行，算太慢了
# x0 = [3.6,21,111,30,12,10,10]
# df = TwiceDifferentiable(feval, g!, x0)

# lx = [0.0 0 -500 0 0 0 0]; ux = [100.0 100 500 τ τ τ 100]
# lc = [-Inf]; uc = [60]
# con_c!(c, x) = (c[1] = x[4]+x[5]+x[6]; c)

# function con_jacobian!(J, x)
# end
# function con_h!(h, x, λ)
# end
# dfc = TwiceDifferentiableConstraints(con_c!,con_jacobian!, con_h!,lx, ux, lc, uc)
# @time res = optimize(df, dfc, x0, IPNewton())

In [101]:
x0 = [3.6,21,111,30,10]
# @time res=optimize(feval,x0,ParticleSwarm(),Optim.Options(iterations=2))
@time res=optimize(feval,g!,x0,LBFGS(),Optim.Options(x_tol=1e-4))

g!  0.31120887615229237 0.31120887615228177 g!  0.3161743428175361 0.3161743428175491 g!  0.3143510957009348 0.31435109570090214 g!  0.31285277931998823 0.31285277931996136 g!  0.31211816383089075 0.312118163830899 g!  0.3116645957203267 0.31166459572032457 g!  0.3114367874322755 0.31143678743229186 g!  0.31132281770265846 0.31132281770267767 g!  0.3112658400566403 0.3112658400566221 g!  0.3112373559696706 0.31123735596969276 g!  0.31122311547530473 0.311223115475295 g!  0.31121599566091784 0.31121599566093966 g!  0.31121243586750236 0.31121243586748953 g!  0.31121065600007997 0.3112106560001092 g!  0.3112097660736538 0.3112097660736591 g!  0.31120932111237937 0.3112093211123697 g!  0.31120909863215634 0.31120909863215 g!  0.3112092098723304 0.3112092098723273 g!  0.3112091542520972 

LoadError: [91mInterruptException:[39m

In [96]:
Optim.minimum(res)

0.3058730736423082

In [97]:
se1=Optim.minimizer(res)

7-element Array{Float64,1}:
   6.311206222991089
  21.91514612385113
 111.57674188453886
  30.369023652636855
  14.645699600483235
  12.288078054530429
  11.267895218133235

In [102]:
#还有什么可以做呢
#冷却带宽的探究，冷却带宽也可以优化
#想要每一段的冷却结果都是最低，评价函数，加权形式？